# Example #1 - pricing a straddle

Using some more complex (but still simple) operations, we can approximate the price of an ATMF straddle.

$$ STRADDLE_{ATMF} \approx \frac{2}{\sqrt{2\pi}} F \times \sigma \sqrt(T) $$
$$ \sigma = implied volatility $$
$$ T = time-to-maturity $$
$$ F = forward of the underlier $$

Let's start with defining the straddle's implied volatility and time-to-maturity. Note, we will assume F is equal to 1 and the straddle price can be scaled accordingly.

In [1]:
vol = 0.2
time = 1.

In [2]:
2. * ( (1 / ( 2 * 3.14 ) ** 0.5 ) * vol * time ** 0.5 )

0.15961737689352445

This is a lot to type again and again if you want to price several straddles, which is really annoying and error prone. Let's define a function for this so that we can use it over and over

In [3]:
def straddlePricer( vol, time ):
    return 2. * ( ( 1. / ( 2 * 3.14 ) ** 0.5 ) * vol * time ** 0.5 )

Notice this doesn't immediately return anything to the output area. Rest assured the function is defined and we can begin using it. Below, we can compare the function's output to the output of the cell above.

In [4]:
print( straddlePricer( 0.2, 1.0 ) )
print( 2. * ( ( 1. / ( 2 * 3.14 ) ** 0.5 ) * vol * time ** 0.5 ) )
print( straddlePricer( 0.2, 1.0 ) == ( 2. * ( ( 1. / ( 2 * 3.14 ) ** 0.5 ) * vol * time ** 0.5 ) ) )

0.15961737689352445
0.15961737689352445
True


Input order doesn't matter as long as we let the function know what we're using as inputs

In [ ]:
print( straddlePricer( time=1.0, vol=0.2 ) )
print( straddlePricer( vol=0.2, time=1.0 ) )

This is nice, but what if I want to default to certain inputs? By setting the initial inputs below we're implictly calling each of these arguments "optional". Initially, we'll make only `time` and optional arguement (input).  

In [ ]:
def straddlePricer( vol, time=1.0 ):
    return 2. * ( ( 1 / ( 2 * 3.14 ) ** 0.5 ) * vol * time ** 0.5 )

In [ ]:
straddlePricer( 0.2 )

Now, we'll make both `vol` and `time` optional.

In [ ]:
def straddlePricer( vol=0.2, time=1.0 ):
    return 2. * ( ( 1 / ( 2 * 3.14 ) ** 0.5 ) * vol * time ** 0.5 )

In other words, we don't need to pass these arguments to call the function. It will use 0.2 for `vol` and 1.0 for `time` by default unless instructed otherwise.

In [ ]:
straddlePricer()

In [ ]:
straddlePricer( 0.22 )

Notice, there's π in the denominator of the straddle price formula, but the value we used above (3.14) is an rough approximation. Is there a more precise value we could use? Yes, we can use a library called `numpy`. Let's import it first below.

In [ ]:
import numpy

You can access functions of numpy by entering `numpy.xxxxx`, where `xxxxx` is the function you would like to use. `numpy`'s implementation of `pi` is simply `numpy.pi`.

In [ ]:
numpy.pi

Typing `numpy` over and over again can get pretty tedious. Let's make it easier for ourselves by abbreviating the name. Python convention for `numpy` abbreviation is `np`.

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
np.pi

`numpy` also has a handy square root function (`np.sqrt`)

In [ ]:
np.sqrt( 4 )

Let's incorporate `np.pi` and `np.sqrt` into our simple straddle pricer to make things a little more precise and easier to read.

In [ ]:
def straddlePricer( vol=0.2, time=1.0 ):
    return 2. * ( ( 1 / np.sqrt( 2 * np.pi ) ) * vol * np.sqrt( time ) )

straddlePricer()

Let's see what the difference is between our original implementation and our new and improved implemenation.

In [ ]:
straddlePricer() - ( 2. * ( ( 1 / ( 2 * 3.14 ) ** 0.5 ) * vol * time ** 0.5 ) )

In this case, the difference in precision and readability isn't huge, but that difference can be valuable at times. In addition to the functionality above, `numpy` can do a lot of other things. For instance, we can generate some random numbers.

In [ ]:
np.random.rand()

Is there a way to see what functions are available? Yes, just tab after `np.`

In [ ]:
#np.

Alternatively, we can call `dir` on `np` to see what is included.

In [ ]:
dir(np)

Continuing with the prior example of pricing our straddle, we can also price the straddle using the Monte Carlo method. We need to generate a normally distributed set of random numbers to simulate the asset's movement through time.

In [ ]:
def straddlePricerMC(vol=0.2, time=1.0, mcPaths=100):
    dailyVol = vol / np.sqrt( 252. )
    resultSum = 0
    for p in range( mcPaths ):
        resultSum += np.abs( np.prod( ( 1 + np.random.normal( 0, dailyVol, int( round( time * 252 ) ) ) ) ) - 1 )
    return resultSum / mcPaths

straddlePricerMC()

There's a lot of new things going on here. Let's unpack it one line at a time.

We know the variance scales linearly with time, so we can either

1. divide the variance by time and take the square root to get a daily volatility, or
2. take the square root of variance (volatility) and divide by the root of time
    
Generally, the latter is clearer and simpler to understand since we typically think in vol terms, but you are free to use whichever method you want.

In [ ]:
# Option #1 above
np.sqrt( vol ** 2 / 252 )

In [ ]:
# Comparing the two methods
vol = 0.2
var = vol ** 2
sqrtVarOverTime = np.sqrt( var / 252 )
volOverSqrtTime = vol / np.sqrt( 252 )
valuesEqual = np.isclose( sqrtVarOverTime, volOverSqrtTime )
print( f'sqrtVarOverTime = {sqrtVarOverTime}\nvolOverSqrtTime = {volOverSqrtTime}\nAre they close? {valuesEqual}' )

The next line isn't super exciting, but we set the default value of our cumulative sum to be 0. So we're just defining resultSum and setting it equal to 0. If we don't do this we'll get an error.

In [ ]:
resultSum = 0

Next we have a loop. There are different types of loops we can use. Here we use a `for` loop, which says "iterate over each element in `range(mcPaths)`". But wait...what's `range(mcPaths)`? `range` is a native python function that will return an iterator over a list of ints starting at 0 and going to x-1.

In [ ]:
range10 = range( 10 )
lst = list( range10 )
print( lst )
print( len( lst ) )

In our case, we don't really want to do anything with `p`, so it is good practice to set it to `_`. We just want to iterate through the loop `mcPaths` times. In the default case, the function runs through the loop 100 times.

In [ ]:
def straddlePricerMC( vol=0.2, time=1.0, mcPaths=100 ):
    dailyVol = vol / np.sqrt( 252. )
    resultSum = 0
    for _ in range( mcPaths ):
        resultSum += np.abs( np.prod( 1 + ( np.random.normal( 0, dailyVol, int( round( time * 252 ) ) ) ) ) - 1 )
    return resultSum / mcPaths

straddlePricerMC()

To unpack what the function does at each iteration of the loop, let's unpack this one step at a time. We start with the innermost function call and work backwards from there. Let's ask for help to see what the `np.random.normal` method actually does. Thankfully, there are two handy ways to see a function's documentation.

1. help
2. ?

In [ ]:
help(np.random.normal)
# np.random.normal?

Ok, so we know from the help function that the `np.random.normal` method takes three optional inputs: mean, standard deviation, and size of the array to generate multiple random numbers. It defaults to a distribution with a mean of zero and a standard deviation of 1, returning only 1 random number.

In [ ]:
np.random.normal()

Below we're going to call this method with a mean of zero (no drift) and a standard deviation of our daily vol, so that we can generate multiple days of returns. Specifically, we ask to generate the number of days to maturity.

In [ ]:
time = 1
nDays = time * 252
dailyVol = vol / np.sqrt( 252. )
print( nDays )

np.random.normal( 0, dailyVol, nDays )

Now, given we have an asset return timeseries, how much is a straddle worth? We're interested in the terminal value of the asset and because we assume the straddle is struck ATM, we can just take the absolute value of the asset's deviation from the initial value (in this case, 1)

In [ ]:
np.random.seed( 42 ) # guarantee the same result from the two random series

returns = np.random.normal( 0, dailyVol, time * 252 )
priceAtMaturity = np.prod( 1 + returns )
changeAtMaturity = priceAtMaturity - 1
absChangeAtMaturity = np.abs( changeAtMaturity )
print( absChangeAtMaturity )

# all together in one line
np.random.seed( 42 )
print( np.abs( np.prod( 1 + ( np.random.normal( 0, dailyVol, time * 252 ) ) ) - 1 ) )

Let's take a closer look at what we did above. This time, we're going to utilize another two libraries called pandas and perspective to make our life a little easier.

In [ ]:
import pandas as pd
from perspective import psp

simulatedAsset = pd.DataFrame( np.random.normal( 0, dailyVol, time * 252 ) + 1, columns=['return'] )
simulatedAsset['price'] = ( 1 * simulatedAsset['return'] ).cumprod()
psp( simulatedAsset )

The `for` loop ultimately just does the above for `mcPaths` number of times, and we ultimately take the average of the paths to find the expected value of the straddle.

In [ ]:
mcPaths = 100
resultSum = 0.
for _ in range(mcPaths):
    resultSum += np.abs( np.prod( 1 + np.random.normal( 0., dailyVol, time * 252 ) ) - 1 )
print( resultSum / mcPaths )

This price is pretty close to the price from our original pricer. More paths should help get us even closer.

In [ ]:
straddlePricerMC(mcPaths=2000)

2000 paths is a lot, but it looks like we're still not converging to the original price. If we add more paths there is a tradeoff with compute time. Luckily, Jupyter has made it really easy to see how fast our function is.

In [ ]:
%timeit straddlePricerMC(mcPaths=2000)

That's pretty fast. we can do a lot more paths.

In [ ]:
print(f"1 path: {straddlePricerMC(mcPaths=1)}")
print(f"2000 path: {straddlePricerMC(mcPaths=2000)}")
print(f"5000 path: {straddlePricerMC(mcPaths=5000)}")
print(f"10000 path: {straddlePricerMC(mcPaths=10000)}")
print(f"100000 path: {straddlePricerMC(mcPaths=100000)}")
print(f"Closed form approximation: {straddlePricer()}")

Can we improve the above MC implementation? Of course! We can generate our random asset series in one go. Remember the `size` argument of the `np.random.normal` function

In [ ]:
nDays = time * 252
size = (nDays, 15)
simulatedAsset = pd.DataFrame(np.random.normal(0, dailyVol, size))
simulatedAsset = (1 + simulatedAsset).cumprod()

simulatedAsset.tail()

Cool!...Let's visualize by plotting it with matplotlib.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')

fig = plt.figure(figsize=(8,6))
ax = plt.axes()
_ = ax.plot(simulatedAsset)

So let's incorporate that into a `pandas` version of the MC pricer.

In [ ]:
def straddlePricerMCWithPD(vol=0.2, time=1, mcPaths=100000):
    dailyVol = vol / ( 252 ** 0.5 )
    randomPaths = pd.DataFrame( np.random.normal( 0, dailyVol, ( time*252, mcPaths ) ) )
    price = ( ( 1 + randomPaths ).prod() - 1 ).abs().sum() / mcPaths
    return price

straddlePricerMCWithPD()